In [1]:
import pandas as pd
import seaborn as sns
from glob import glob
from matplotlib.font_manager import fontManager, FontProperties

path = "/usr/share/fonts/opentype/linux-libertine/LinBiolinum_R.otf"
fontManager.addfont(path)
sns.set_theme(style="ticks", font=FontProperties(fname=path).get_name(), font_scale=1.2)

In [6]:
data = pd.concat([pd.read_csv(data, names=[
    "Variant", "#Replica/Region", "#Client/Region", "#PutClient/Region", "Throughput", "Latency"
]) for data in glob('cops-*.txt')], ignore_index=True)
data["Put %"] = data["#PutClient/Region"] / 2 / data["#Client/Region"] * 100
data

,Variant,#Replica/Region,#Client/Region,#PutClient/Region,Throughtput,Latency,Put %
0,Untrusted,1,100,60,66590.970,0.007596,30.0
1,Untrusted,1,100,40,90839.900,0.005564,20.0
2,Untrusted,1,100,4,214341.830,0.002333,2.0
3,Untrusted,1,100,1,246828.300,0.002026,0.5
4,Untrusted,1,100,80,54223.188,0.009375,40.0
5,Untrusted,1,100,8,186468.660,0.002683,4.0
6,Untrusted,1,100,2,233349.750,0.002144,1.0
7,Untrusted,1,100,20,138210.050,0.003634,10.0


In [7]:
sns.barplot(data=data,
    x="Put %", y="Throughput", hue="Variant")

ValueError: Could not interpret input 'Throughput'